# Анализ растров [norm.]diff1516.N 

In [ ]:
import numpy as np

import utilites

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
    get_gisbase_path
)

import grasslib
reload(grasslib)

from grasslib import GRASS

In [ ]:
grs = GRASS(gisbase=get_gisbase_path(), 
            dbase=get_grassdata_path(), 
            location=get_location_name(),
            mapset='klsvd'
)

In [ ]:
# inputs = ['diff1615.%d@basemaps'%i for i in range(1, 12)]
inputs = ['norm.diff1615.%d@basemaps'%i for i in range(1, 12)]
print inputs
output = 'wint1516.alrm@umd_alarm'

In [ ]:
grs.grass.run_command('g.region', region="all_scenes@landsat", res=240)

In [ ]:
from sklearn.model_selection import train_test_split

X_data = grs.rasters_to_array(inputs)
Y_data = grs.rasters_to_array([output]).ravel()

# In output raster '1' is 'alarm', nan is 'not alarm'
Y_data = np.nan_to_num(Y_data)

In [ ]:
print X_data.shape
print Y_data.shape
print np.count_nonzero(Y_data)

In [ ]:
# Сортируем так чтобы объекты-рубки оказались в конце списка, 
# чтобы на графиках их было видно - они будут отрисовываться после объектов-нерубок
Y_inds = Y_data.argsort()

X_data_sorted = X_data[Y_inds]
Y_data_sorted = Y_data[Y_inds]

In [ ]:
from matplotlib.colors import ListedColormap

%pylab inline

# Аттрибутов у нас 11.
# i, j - инексы аттрибутов которое хотим отразить на графике
def print_plot(i, j, sorted_x, sorted_y):
    
    colors = ListedColormap(['red', 'blue'])
    
    pylab.figure(figsize=(16,12))
    pylab.scatter(
        sorted_x[ : , i],
        sorted_x[ : , j],
        c = sorted_y[ : ],
        cmap = colors)
    pylab.title('Attributes %d %d' % (i, j))
    pylab.xlabel('attr %d' % i)
    pylab.ylabel('attr %d' % j)
    plt.colorbar()
    pylab.show()

In [ ]:
for i in range(11):
    for j in range(11):
        if j > i:
            print_plot(i, j, X_data_sorted, Y_data_sorted)

## Те же действия внутри маски лесов

Замаскируем все леса. Что лежит вне маски лесов, становится null. (У меня тенденция забывать удалять маску после использования. Поэтому я оберну все в try-finally)

In [ ]:
try:
    grs.grass.run_command('r.mapcalc', expression="MASK = forest.mask.60@treecover")
    X_data = grs.rasters_to_array(inputs)
    Y_data = grs.rasters_to_array([output]).ravel()
finally:
    # grs.grass.run_command('g.remove', type='rast', name="MASK", flags='f')
    grs.grass.run_command('r.mask', flags='r')
    
# In output raster '1' is 'alarm', nan is 'not alarm'
Y_data = np.nan_to_num(Y_data)

In [ ]:
print X_data.shape
print Y_data.shape
print np.count_nonzero(Y_data)

In [ ]:
# Сортируем так чтобы объекты-рубки оказались в конце списка, 
# чтобы на графиках их было видно - они будут отрисовываться после объектов-нерубок
Y_inds = Y_data.argsort()

X_data_sorted = X_data[Y_inds]
Y_data_sorted = Y_data[Y_inds]

In [ ]:
for i in range(11):
    for j in range(11):
        if j > i:
            print_plot(i, j, X_data_sorted, Y_data_sorted)

In [ ]:
inputs = ['diff1615.%d@basemaps'%i for i in range(1, 12)]
# inputs = ['norm.diff1615.%d@basemaps'%i for i in range(1, 12)]
print inputs
output = 'wint1516.alrm@umd_alarm'

In [ ]:
try:
    grs.grass.run_command('r.mapcalc', expression="MASK = forest.mask.60@treecover")
    X_data = grs.rasters_to_array(inputs)
    Y_data = grs.rasters_to_array([output]).ravel()
finally:
    # grs.grass.run_command('g.remove', type='rast', name="MASK", flags='f')
    grs.grass.run_command('r.mask', flags='r')
    
# In output raster '1' is 'alarm', nan is 'not alarm'
Y_data = np.nan_to_num(Y_data)

In [ ]:
print X_data.shape
print Y_data.shape
print np.count_nonzero(Y_data)

# Сортируем так чтобы объекты-рубки оказались в конце списка, 
# чтобы на графиках их было видно - они будут отрисовываться после объектов-нерубок
Y_inds = Y_data.argsort()

X_data_sorted = X_data[Y_inds]
Y_data_sorted = Y_data[Y_inds]

In [ ]:
for i in range(11):
    for j in range(11):
        if j > i:
            print_plot(i, j, X_data_sorted, Y_data_sorted)

## Оценка плотностей

Графики показывают то, что алармы имеют тенденцию накапливаться около нуля, а не-алармы разбросаны чуть больше. Оценим плотности распределения точек для алармов и не-алармов в виде гауссовых смесей.

In [ ]:
# inputs = ['diff1615.%d@basemaps'%i for i in range(1, 12)]
inputs = ['norm.diff1615.%d@basemaps'%i for i in range(1, 12)]
print inputs
output = 'wint1516.alrm@umd_alarm'

try:
    grs.grass.run_command('r.mapcalc', expression="MASK = forest.mask.60@treecover")
    X_data = grs.rasters_to_array(inputs)
    Y_data = grs.rasters_to_array([output]).ravel()
finally:
    # grs.grass.run_command('g.remove', type='rast', name="MASK", flags='f')
    grs.grass.run_command('r.mask', flags='r')
    
# In output raster '1' is 'alarm', nan is 'not alarm'
Y_data = np.nan_to_num(Y_data)

In [ ]:
X_alarm = X_data[Y_data==1]
print X_alarm.shape
X_alarm = X_alarm[~np.isnan(X_alarm).any(axis=1), :]
print X_alarm.shape

In [ ]:
X_nonalarm = X_data[Y_data==0]
print X_nonalarm.shape
X_nonalarm = X_nonalarm[~np.isnan(X_nonalarm).any(axis=1), :]
print X_nonalarm.shape

In [ ]:
from sklearn import mixture

lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
for n_components in n_components_range:
    # Fit a Gaussian mixture with EM
    gmm = mixture.GaussianMixture(n_components=n_components,
                                  covariance_type='full')
    gmm.fit(X_alarm)
    bic.append(gmm.bic(X_alarm))
    if bic[-1] < lowest_bic:
        lowest_bic = bic[-1]
        best_gmm_alarm = gmm

bic

In [ ]:
lowest_bic = np.infty
bic = []
n_components_range = range(1, 20)
for n_components in n_components_range:
    # Fit a Gaussian mixture with EM
    gmm = mixture.GaussianMixture(n_components=n_components,
                                  covariance_type='full')
    gmm.fit(X_nonalarm)
    bic.append(gmm.bic(X_nonalarm))
    if bic[-1] < lowest_bic:
        lowest_bic = bic[-1]
        best_gmm_nonalarm = gmm

bic

In [ ]:
print best_gmm_alarm.means_
# print best_gmm_alarm.covariances_

In [ ]:
print best_gmm_nonalarm.means_
# print best_gmm_nonalarm.covariances_

## Исходные каналы (не разности)

In [ ]:
inputs15 = ['composite15.%d@basemaps'%i for i in range(1, 12)]
inputs16 = ['composite16.%d@basemaps'%i for i in range(1, 12)]
inputs = inputs15 + inputs16

# inputs = ['norm.diff1615.%d@basemaps'%i for i in range(1, 12)]
print inputs
output = 'wint1516.alrm@umd_alarm'

try:
    grs.grass.run_command('r.mapcalc', expression="MASK = forest.mask.60@treecover")
    X_data = grs.rasters_to_array(inputs)
    Y_data = grs.rasters_to_array([output]).ravel()
finally:
    # grs.grass.run_command('g.remove', type='rast', name="MASK", flags='f')
    grs.grass.run_command('r.mask', flags='r')
    
# In output raster '1' is 'alarm', nan is 'not alarm'
Y_data = np.nan_to_num(Y_data)

print X_data.shape
print Y_data.shape
print np.count_nonzero(Y_data)

# Сортируем так чтобы объекты-рубки оказались в конце списка, 
# чтобы на графиках их было видно - они будут отрисовываться после объектов-нерубок
Y_inds = Y_data.argsort()

X_data_sorted = X_data[Y_inds]
Y_data_sorted = Y_data[Y_inds]

In [ ]:
for i in range(11):
    j = i + 11
    print_plot(i, j, X_data_sorted, Y_data_sorted)